In [ ]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import date, timedelta
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
sns.set_style(style='darkgrid')

In [ ]:
warnings.filterwarnings(action='ignore')

In [ ]:
end_date = date.today().strftime('%Y-%m-%d')

start_date = date.today() - timedelta(days=365)
start_date = start_date.strftime('%Y-%m-%d')

df = yf.download(tickers='GOOG',
                 start=start_date,
                 end=end_date,
                 progress=False)

df['Date'] = df.index
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

In [ ]:
df = df[['Date', 'Close']]
df.head()

In [ ]:
px = 1 / plt.rcParams['figure.dpi']
plt.figure(figsize=(1200*px, 600*px))
plt.plot(df['Date'], df['Close'])
plt.show()

In [ ]:
result = seasonal_decompose(x=df['Close'], model='multiplicative', period=30)

result.plot()
plt.show()

In [ ]:
pd.plotting.autocorrelation_plot(df["Close"])
plt.show()

In [ ]:
plot_pacf(df["Close"], lags=100)

In [ ]:
p, d, q = 5, 1, 2

model_arima = sm.tsa.ARIMA(df["Close"], order=(p, d, q))

model_arima = model_arima.fit()

In [ ]:
print(model_arima.summary())

In [ ]:
arima_predictions = model_arima.predict()
print(arima_predictions)

In [ ]:
model_sarimax = sm.tsa.statespace.SARIMAX(df['Close'],
                                          order=(p, d, q),
                                          seasonal_order=(p, d, q, 12))

model_sarimax = model_sarimax.fit()

In [ ]:
print(model_sarimax.summary())

In [ ]:
sarimax_predictions = model_sarimax.predict(len(df), len(df) + 10)
print(sarimax_predictions)

In [ ]:
df["Close"].plot(legend=True, label="Training Data", figsize=(12, 6))
sarimax_predictions.plot(legend=True, label="Predictions")